In [1]:
%matplotlib ipympl
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import numpy as np
import scipy.io as sp
from mpl_toolkits.mplot3d import Axes3D
import pickle
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
#Loading Dataset
dataSet = sp.loadmat('breastw.mat')
#Splitting dataset
Xtraining = dataSet['X'] #in this first run we use the whole dataset
X = Xtraining
#Labels
y = dataSet['y']

In [3]:
#Cross Validation on default settings
cv = StratifiedKFold(10)
model = KMeans(n_clusters = 2)#, init = "k-means++", n_init = 100, max_iter =800, algorithm="full")
cvs = cross_val_score(model, X, y, cv=cv,scoring='f1')
print(cvs)
print(np.mean(cvs))
print(np.std(cvs))
print(np.max(cvs))



[0.84444444 0.18421053 0.96       0.91666667 0.97959184 0.95833333
 0.05714286 0.9787234  0.02898551 0.        ]
0.5908098576139481
0.4310849973222447
0.9795918367346939


In [ ]:
#Calculating each model by iterating every parameter. Results are stored in a dictionnary compressed and save using the "pickle" library for later postprocessing
mat = np.empty((0,50))
Counter = 0
keys = ["Init","N_init","Max_iter","Algorithm","cv_score","F1_score_moyen","Ecart_type","F1_score_min","F1_score_max"]
dict = {"Init":[],"N_init":[],"Max_iter":[],"Algorithm":[],"cv_score":[],"cv_mean":[],"cv_SD":[],"cv_min":[],"cv_max":[]}
for Init in ["k-means++", "random"]:
    for N_init in range(1,100,10):
        for Max_iter in range(200,500,10):
            for Algorithm in ["full", "elkan"]:
                        
                model = KMeans(n_clusters = 2, init = Init, n_init = N_init, max_iter = Max_iter, algorithm=Algorithm)
                
                cv_score = cross_val_score(model, X, y, cv=cv,scoring='f1')
                cv_mean = np.mean(cv_score)
                cv_SD = np.std(cv_score)
                cv_min = np.min(cv_score)
                cv_max = np.max(cv_score)

                dictAttributes = [Init, N_init,Max_iter,Algorithm,cv_score,cv_mean,cv_SD,cv_min,cv_max]
                for keyidx in range(len(keys)):
                    dict[keys[keyidx]].append(dictAttributes[keyidx])
                
                #print(dict)
                Counter += 1
                print(Counter)

with open('DictCV.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(dict, f, protocol=pickle.HIGHEST_PROTOCOL)
                

In [ ]:
# Retrieving stored dictionnary
with open('DictCV.pkl','rb') as f:  # Python 3: open(..., 'rb')
    CompareResults = pickle.load(f)


In [ ]:
#Creating a pandas Dataframe from results dictionnary
DataFrame = pd.DataFrame(data=CompareResults)

In [29]:
Opt = DataFrame.sort_values(by="F1_score_moyen", ascending=False)
Opt = Opt.drop(['cv_score','F1_score_min','F1_score_max'],axis=1);
Best = Opt.head(14)
Best


,Init,N_init,Max_iter,Algorithm,F1 score moyen,ecart type
271,k-means++,41,350,elkan,0.940227,0.055778
344,k-means++,51,420,full,0.940227,0.055778
195,k-means++,31,270,elkan,0.940227,0.055778
306,k-means++,51,230,full,0.940227,0.055778
332,k-means++,51,360,full,0.940227,0.055778
141,k-means++,21,300,elkan,0.940227,0.055778
505,k-means++,81,320,elkan,0.940227,0.055778
506,k-means++,81,330,full,0.940227,0.055778
423,k-means++,71,210,elkan,0.940227,0.055778
471,k-means++,71,450,elkan,0.940227,0.055778
